In [137]:
%load_ext autoreload
%autoreload 2
import sys
import pandas as pd
import pickle
import itertools 
sys.path.append("../")
from title_maker_pro import datasets
from collections import OrderedDict
import copy

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
dataset_path = "/mnt/evo/projects/title-maker-pro/data/urban_dictionary_words.pickle"
with open(dataset_path, 'rb') as f:
    dataset = pickle.load(f)

In [80]:
blacklist = datasets.Blacklist.load("/mnt/evo/projects/title-maker-pro/models/blacklist.pickle")

In [21]:
pd_dataset = pd.DataFrame(
    (
        (d.word, d.meaning, d.examples[0], d.upvotes, d.downvotes, d.creation_epoch) 
        for d in itertools.chain.from_iterable(e.definitions for e in dataset.values())
    ),
    columns=["word", "meaning", "example", "upvotes", "downvotes", "creation_epoch"]
)

In [99]:
def cut(original, f, name):
    n = original[f]
    print(f"{name} cut by {100 * len(n) / (len(original)):.2f}% ({len(original)} -> {len(n)})")
    return n

In [103]:
t = pd_dataset.copy()
t = cut(t, ~(pd_dataset["word"].apply(blacklist.contains)), name="blacklist")
t = cut(t, ((pd_dataset["example"].str.len() + pd_dataset["meaning"].str.len() + pd_dataset["word"].str.len()) < 250), name="length")
t = cut(t, (pd_dataset["upvotes"] >= 4), name="upvotes")

blacklist cut by 49.63% (2961824 -> 1469932)


/home/tdimson/anaconda3/envs/company_makeup/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


length cut by 65.20% (1469932 -> 958444)
upvotes cut by 47.83% (958444 -> 458387)


In [148]:
cleaned_dataset = OrderedDict()
i = 0
num_defns = 0
for k, ud_word in dataset.items():
    good_defns = []
    for d in ud_word.definitions:
        if i in valid_indexes:
            good_defns.append(d)
            num_defns += 1
        i += 1
    
    if good_defns:
        new = copy.deepcopy(ud_word)
        new.definitions = good_defns
        cleaned_dataset[k] = new

In [150]:
cleaned_dataset_path = "/mnt/evo/projects/title-maker-pro/data/urban_dictionary_250_cleaned.pickle"
with open(cleaned_dataset_path, "wb") as f:
    pickle.dump(cleaned_dataset, f, pickle.HIGHEST_PROTOCOL)